# Measuring cuboids


In [9]:
# You can only fill in a file with a .ply extension
fileName = '1673533868_-0.06753296,-0.18555829,-0.3208556'

top_plane_point_x = -0.06753296
top_plane_point_y = -0.18555829
top_plane_point_z = -0.3208556

## 1. Importing libraries and data

In [10]:
import open3d as o3d

# remove outliers
cloud = o3d.io.read_point_cloud(fileName + ".ply")

[Open3D WARNING] Read PLY failed: unable to open file: 1673533936_-0.14547378,-0.42771333,-0.43153822.ply


RPly: Unable to open file


## 2. Statistical Outlier Filter

In [11]:
cloud, ind = cloud.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
o3d.io.write_point_cloud(f"{fileName}_1_statOutlierRemoval.ply", cloud)

[Open3D WARNING] Write PLY failed: point cloud has 0 points.


False

## 3. Radius Outlier Removal

In [12]:
cloud, ind = cloud.remove_radius_outlier(nb_points=10, radius=0.01)
o3d.io.write_point_cloud(f"{fileName}_2_removeRadiusOutlier.ply", cloud)

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.


False

## 4. Voxel Down Sample

In [13]:
cloud = cloud.voxel_down_sample(voxel_size=0.005)
o3d.io.write_point_cloud(f"{fileName}_3_voxelDownSample.ply", cloud)

[Open3D WARNING] Write PLY failed: point cloud has 0 points.


False

# 5. Floor removal
\subsubsection{Verwijderen van de vloer} 

De volgende stap die het algoritme moet uitvoeren is het verwijderen van de vloer. Alle punten die samen het vlak vormen waar de cuboid opstaat tijdens de meting worden als de vloer beschouwd. Om dit vlak te herkennen wordt de RANSAC-methode gebruikt. De implementatie van 
\\
\textcite{Mariga_pyRANSAC-3D_2022}, pyRANSAC3D, wordt in het algoritme van deze paper gebruikt. Verder wordt ook de NumPy library van \textcite{harris2020array} gebruikt. NumPy is een Python-bibliotheek voor het werken met numerieke data. Het biedt onder andere efficiënte implementaties van multi-dimensionale arrays en wiskundige functies om deze arrays te manipuleren. 

In [14]:
import numpy as np
# get a random point from the top_plane and fill in the x, y and z coordinate.
# top_plane_point_x = 
# top_plane_point_y = 
# top_plane_point_z = 

normal = np.array([0, 1, 0])
top_plane_point = np.array([top_plane_point_x, top_plane_point_y, top_plane_point_z])
top_plane_point_product = -np.dot(normal, top_plane_point)
points = np.asarray(cloud.points)

threshold = 0.02

# points_in_plane = points[(np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold]
points_not_in_plane = points[~((np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold)]


Als we de vloer willen verwijderen zullen we opzoek gaan naar een vlak. Hierbij kan het probleem optreden dat de bovenkant van de doos herkent wordt als het vloervlak en dus verwijderd wordt. Om dit te voorkomen zullen we de bovenvlak en 2cm eronder van de doos tijdelijk uit het pointcloud beeld verwijderen. Tijdens deze stap is er input van de gebruiker vereist. Het is de bedoeling dat de variabelen \texttt{top_plane_point_x}, \texttt{top_plane_point_y} en \texttt{top_plane_point_z} ingevuld worden als de coordinaten van een punt dat zich op het bovenste vlak van de cuboid bevindt. Dit punt zal gebruikt worden om het bovenvlak te vinden en de punten die tot 2 cm daaronder liggen.

\texttt{np.asarray(cloud.points)}: Deze functie converteert de lijst van punten in de pointcloud naar een twee-dimensionale numpy array, waarbij elke rij de coordinaten van een punt bevat. Dit is handig voor het werken met de punten in de puntewolk, omdat numpy arrays gemakkelijk te indexeren zijn en snelle numerieke berekeningen mogelijk maken.

\texttt{threshold}: Hoeveel afstand een punt maximaal mag hebben tot het bovenste vlak van de cuboid om als inlier beschouwd te worden en dus mee verwijdert te worden. Dit werd in het algoritme ingesteld op \teksttt{0.02}, dit is dus 2 cm. Hiermee moet rekeningen gehouden worden bij het opmeten van de doos, de 2 cm moet namelijk terug opgeteld worden bij de hoogte van de doos.

Het bovenstevlak wordt gedefinieerd door een normaalvector, normaal, en een punt op het vlak, \texttt{top_plane_point}.\\texttt{top_plane_point_product} wordt berekend als de ontkenning van het puntproduct van de normaal en het punt op het vlak.

\texttt{points_not_in_plane}: Een numpy array die de punten bevat van de pointcloud die niet binnen de gespecificeerde afstand (bepaald door de threshold) van het bovenste vlakliggen. Met andere woorden, alle punten behalve het bovenste vlak van de doos en de punten die binnen de 2cm ervan liggen.

In [15]:
# Visualize
v_points_in_plane = np.arange(points.shape[0])[(np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold]
v_points_not_in_plane = np.arange(points.shape[0])[~((np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold)]


visual = cloud.select_by_index(v_points_not_in_plane, invert=False)
visual.paint_uniform_color([0, 1, 0])
visual2 = cloud.select_by_index(v_points_in_plane, invert=False)
visual2.paint_uniform_color([0, 0, 1])
o3d.io.write_point_cloud(f"{fileName}_x_pointsNotIP.ply", visual)
o3d.io.write_point_cloud(f"{fileName}_x_pointsIP.ply", visual2)
o3d.io.write_point_cloud(f"{fileName}_x_combined.ply", visual + visual2)

# TEKEN tap point
# mesh = o3d.geometry.TriangleMesh.create_sphere(radius=0.02)
# mesh.translate(top_plane_point)

# o3d.visualization.draw_geometries([mesh, cloud])

[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.


False

In [16]:
import pyransac3d as pyrsc
# convert cloud to numpy array
floor_plane = pyrsc.Plane()
floor_cutting = 0.02

best_eq, best_inliers = floor_plane.fit(points_not_in_plane, floor_cutting, 1000)

# Points that do not belong to the floor found floorplane
points_no_floor = points[~((np.abs(np.dot(best_eq[0:3], points.T) + best_eq[3])) < threshold)]
# Points that do not belong to the floor found floorplane and are not below the floor
points_not_below_floor = points_no_floor[~(points_no_floor[:, 1] < (-best_eq[3] / best_eq[1]))] 

newCloud = o3d.geometry.PointCloud()
newCloud.points = o3d.utility.Vector3dVector(np.array(points_not_below_floor))
# o3d.visualization.draw_geometries([newCloud])
o3d.io.write_point_cloud(f"{fileName}_4_floorPlane.ply", newCloud)

ValueError: Sample larger than population or is negative


\texttt{pyrsc.Plane()}: Plane class van de PyRANSAC3D library wordt geïmporteerd en er een \texttt{Plane} object wordt gemaakt.
\texttt{floor\_cutting}: Een parameter die aangeeft hoe ver een punt maximaal van het vlak kan zijn verwijderd en nog als een inlier (dus als onderdeel van het vlak) beschouwd wordt.\\

\texttt{floor\_plane.fit(points_not_in_plane, floor\_cutting, 1000)}: De \texttt{fit} method geeft de vergelijking van het best passende vlak en de indices van de punten die als inliers beschouwd worden terug door RANSAC toe te passen. De \texttt{points_not_in_plane} parameter is de lijst van punten zonder bovenvlak van de doos. De \texttt{floor\_cutting} parameter geeft aan hoe ver een punt maximaal van het vlak kan zijn en nog als een inlier (dus als onderdeel van het vlak) beschouwd wordt. De \texttt{1000} parameter geeft aan hoeveel iteraties het RANSAC algoritme maximaal mag doen.\\

\texttt{points\_no\_floor}: is een numpy-array die de punten bevat van de pointcloud die niet binnen het gevonden vloervlak liggen.\\

\texttt{points\_not\_below\_floor}: Een numpy-array die de punten bevat van de pointcloud die niet onder of binnen het vloervlak liggen.\\


<!-- \texttt{floor\_plane.get\_oriented\_bounding\_box():} Deze methode berekent de georiënteerde bounding box die past rond de punten die behoren tot \texttt{floor\_plane} object en slaat het resultaat op in het obb object. \texttt{obb.color = [0, 0, 1]} zet de kleur van de boundingbox op blauw. \\

\texttt{obb.get\_point\_indices\_within\_bounding\_box(cloud.points)} geeft een lijst van indices van punten in de puntewolk terug die zich binnen de bounding box bevinden. -->

## 6. Isoleren van de cuboid en zijn vlakken

In [ ]:
# Alle punten vinden die in de vorm van een kubus zijn
cuboid = pyrsc.Cuboid()
# best_eq is een numpy matrix van 3 beste vlakkenvergelijking
# best_inliers is een lijst van de indexen van de punten die in de kubus passen
best_eq, best_inliers = cuboid.fit(np.asarray(points_not_below_floor), thresh=0.005)

# VISUALISATIE START
# cuboid_pcl is een pointcloud van de punten die in de kubus passen
# not_cuboid_pcl is een pointcloud van de punten die niet in de kubus passen
cuboid_pcl = newCloud.select_by_index(best_inliers).paint_uniform_color([1, 0, 0])
not_cuboid_pcl = newCloud.select_by_index(best_inliers, invert=True).paint_uniform_color([0, 1, 0])
plane = newCloud.select_by_index(best_inliers).paint_uniform_color([1, 0, 0])

# Join  plane and not plane togheter 
o3d.io.write_point_cloud(f"{fileName}_6_cuboidPcl.ply", cuboid_pcl)
o3d.io.write_point_cloud(f"{fileName}_7_nonCuboidPcl.ply", not_cuboid_pcl)
o3d.io.write_point_cloud(f"{fileName}_8_combinedCuboidAndNonCuboidPcl.ply", cuboid_pcl + not_cuboid_pcl)
# VISUALISATIE EINDE

Nu de vloer verwijderd is uit de pointcloud kunnen we de cuboid proberen te isoleren. De methode \texttt{pyrsc.Cuboid()} wordt gebruikt om een \texttt{cuboid} object aan te maken. De \texttt{fit} methode van het Cuboid object zoekt de beste vergelijking voor 3 vlakken die een complete kubus definiëren. \texttt{best_eq} is een numpy matrix die de vergelijking van de 3 beste vlakken bevat. \texttt{best_inliers} is een numpy array die de punten bevat die binnen de 3 vlakken vallen.

In [ ]:
horizontal_plane_idx = 0
left_plane_idx = 1
right_plane_idx = 2

# Haal de beste vlakkenvergelijkingen op voor de horizontale, linker en rechter vlakken
top_plane = best_eq[horizontal_plane_idx]
left_plane = best_eq[left_plane_idx]
right_plane = best_eq[right_plane_idx]

# VISUALISATIE START

# VISUALISATIE EINDE

De gevonden beste vlakkenvergelijkingen van het horizontale, linker en rechter vlak van de cuboid worden opgehaald en in een variabele opgeslagen.

## 7. Vectoren berekenen aan de hand van algebra

Om een cuboid te draaien zodat een van zijn zijden evenwijdig is aan een van de assen (x, y of z), moeten eerst de normalen van zijn vlakken gevonden worden. Deze normalen bieden een referentiekader voor de cuboid in zijn oorspronkelijke positie en oriëntatie.


In [ ]:
# Van waar komt deze functie? Zelfgeschreven?
def plane_intersect(a, b):
    """
    a, b   4-tuples/lists
           Ax + By +Cz + D = 0
           A,B,C,D in order  

    output is een punt en een vector

p_inter is punt op d eintersectielijn en aXb_vec is de richtingsvector
    """
    a_vec, b_vec = np.array(a[:3]), np.array(b[:3])

    aXb_vec = np.cross(a_vec, b_vec)

    A = np.array([a_vec, b_vec, aXb_vec])
    d = np.array([-a[3], -b[3], 0.]).reshape(3,1)

    p_inter = np.linalg.solve(A, d).T

    return p_inter[0], aXb_vec

// TODO: KORTE UITLEG EERST
// TODO: nalezen, klopt dit?
// Hoe leg ik kort en bondig uit wat een vector is?
Nu we de vergelijking hebben van elk vlak kunnen we via algebraeis berekeningen ook de interesectie van 2 vlakken met elkaar zoeken.



Hier word een functie \texttt{plane\_intersect} gedefiniëerd. Deze neemt twee argumenten op, \texttt{a} en \texttt{b}, die 4-tupels zijn ofwel lijsten die de vergelijkingen van twee vlakken vertegenwoordigen. In dit geval zullen dit de vlaksvergelijkingen zijn, \texttt{floor_plane}, \texttt{left_plane}, \texttt{right_plane}. De functie converteert de a en b 4-tupels/lijsten naar 3-element numpy arrays \texttt{a_vec} en \texttt{b_vec} respectievelijk, waarbij \texttt{a_vec} = \[A, B, C\] en \texttt{b_vec} = \[A, B, C\].

De functie converteert de a en b 4-tupels/lijsten naar 3-element numpy arrays a_vec en b_vec respectievelijk, waarbij a_vec = [A, B, C] en b_vec = [A, B, C].

Vervolgens berekent het het kruisproduct van \texttt{a_vec] en \texttt{b_vec}, dat de richtingsvector geeft van de snijlijn van de twee vlakken, en wijst het toe aan de variabele \texttt{aXb_vec}.

Het creëert een matrix A met de rijen \texttt{a_vec}, \texttt{b_vec} en \texttt{aXb_vec}, en een kolomvector d met elementen \texttt{[-a[3], -b[3], 0}.

\texttt{linalg.solve()}: gaat op zoek naar een punt, \texttt{p\_inter}, dat zowel op vlak \texttt{A} als op vlak \texttt{B} als op de richtingsvector van de snijlijn van \texttt{A} en \texttt{B} ligt. Dit punt wordt geretourneerd als een 1x3 array.

Het snijpunt wordt berekend met de methode linalg.solve van numpy en retourneert het snijpunt van de twee vlakken als een numpy-array van vorm (3,). De functie retourneert ook de richtingsvector van de snijlijn, \texttt{aXb_vec}.

We willen de cuboid opmeten, maar daarvoor moeten we die gelijkstellen met de assen van onze ruimte. We hebben nu enkel de vergelijking van de vlakken, maar een ruimte is gedefinieerd door 3 vectoren. We moeten dus 3 loodrechte ribben van onze kubus extraheren om de nieuwe ruimte op te stellen. Dit doen we met de intersecties, aan de hand van deze methode.


In [ ]:
_, new_y_vector = plane_intersect(left_plane, right_plane)
_, new_x_vector = plane_intersect(top_plane, right_plane)
new_z_vector = np.cross(new_x_vector, new_y_vector)

\texttt{np.cross()}: Deze functie wordt gebruikt om het kruisproduct van twee vectoren te berekenen. Dit kruisproduct van twee vectoren geeft als resultaat een vector die loodrecht staat op beide oorspronkelijke vectoren. De richting van deze resulterende vector kan bepaald worden door de "rechterhandregel". Wanneer de vingers van de rechterhand gekruld zijn in de richting van de eerste vector en de duim wijst in de richting van de tweede vector, is de resulterende vector in de richting van de uitgestrekte duim. In deze code wordt het kruisproduct van new_x_vector en new_y_vector berekend om een vector new_z_vector te verkrijgen, die loodrecht op beide vectoren staat.

(TODO Afbeelding van rechterhandregel toevoegen)

In [ ]:
# normalize vectors
new_x_vector = new_x_vector / np.linalg.norm(new_x_vector)
new_y_vector = new_y_vector / np.linalg.norm(new_y_vector)
new_z_vector = new_z_vector / np.linalg.norm(new_z_vector)

Tijdens deze stap willen we onze gevonden vectoren normaliseren. In de wiskunde verwijst het normaliseren van een vector naar het proces waarbij de lengte van de vector wordt aangepast zodat deze gelijk wordt aan 1. Deze wordt dan ook wel een "eenheidsvector" genoemd. Hiervoor wordt in ons algoritme \texttt{np.linalg.norm()} gebruikt. Deze methode berekent de Euclidische norm (ofwel de L2-norm) van de ingevoerde vector. Dit wordt door numpy berekend als de vierkantswortel van de som van de kwadraten van de elementen van de vector.

Het resultaat van deze methode wordt gedeeld door de vector zelf om een eenheidsvector te bekomen. Deze eenheidsvector wijst nog steeds in dezelfde richting als de originele, maar hij is van lengte 1. Dit is een gebruikelijke bewerking bij vectorberekeningen, odat het ons in staat stelt gemakkelijker vectoren te vergelijken of combineren met elkaar.

# 8. De cuboid roteren
Nu de normalen van alle vlakken van de cuboid gevonden zijn, kunnen we deze gebruiken om de cuboid te draaien of roteren. We willen de cuboid zo roteren dat de x-,y- en z- zijden van onze cuboid  parallel zijn aan de x-,y- en z-assen. We gebruiken de normalen om een nieuw coördinatensysteem voor de cuboid te definiëren. Hierbij zijn de x-,y- en z-vectoren van het nieuwe coördinatensysteem gegeven door de normalen van de vlakken van de cuboid.

Zodra deze normalen zijn gevonden, kunnen we ze gebruiken om een nieuw coördinatensysteem voor de kubus te definiëren, waarbij de x-, y- en z-vectoren van het nieuwe coördinatensysteem worden gegeven door de normalen van de vlakken van de kubus. We kunnen dan een transformatiematrix gebruiken om de kubus in dit nieuwe coördinatensysteem te roteren en uit te lijnen met de x-, y- of z-as.

In [ ]:
cuboid_pcl_points = np.asarray(cuboid_pcl.points)

base_change_matrix = np.array([new_x_vector, new_y_vector, new_z_vector]).reshape(3,3)
new_points = np.dot(base_change_matrix, cuboid_pcl_points.T).T

\teksttt{base_change_matrix}: In deze variabele wordt een 3x3 numpy matrix opgeslagen die gemaakt werd uit de 3 input vectoren. Deze matrix zal later worden gebruikt om de cuboid_pcl_points in de volgende coderegel te roteren.

\teksttt{new_points}: Deze variabele bevat de punten van de geroteerde cuboid. 

\teksttt{np.dot()}: Om de cuboid te roteren wordt numpy gebruikt om het puntproduct te berekenen. Deze bereknt de matrisvermenigvuldigingen van de \teksttt{base_change_matrix} en de getransponeerde \teksttt{cuboid_pcl_points} puntenwolk. Het resultaat hiervan word getransponeerd en dit geeft als resultaat een geroteerde puntenwolk. Deze is georienteerd volgens de oriëntatie van de 3 vectoren van de vlakken. Hierdoor worden deze punten uitgelijnd met de x-, y- of z-as.

In [ ]:
# to pointcloud
new_cuboid_pcl = o3d.geometry.PointCloud()
new_cuboid_pcl.points = o3d.utility.Vector3dVector(new_points)

# # visualize
# o3d.io.write_point_cloud(f"{fileName}_10_newCuboid.ply", new_cuboid_pcl)

# # TODO Waarom hier nog eens radius removal? 
# new_cuboid_pcl, ind = new_cuboid_pcl.remove_radius_outlier(nb_points=16, radius=0.01)
# # visualize
# o3d.io.write_point_cloud(f"{fileName}_11_newCuboidAfterRadiusRemoval.ply", new_cuboid_pcl)


\teksttt{new_cuboid_pcl}: In deze variabele wordt de geroteerde punten van \teksttt{new_points} opgeslagen als een open3d PointCloud object.

In [ ]:
# new_cuboid_pcl_points = np.asarray(new_cuboid_pcl.points)
# max_dist_from_neighbor = 0.02
# points_to_remove = []
# # Filtermethode neighbour filtering?
# # Is deze stap nu nog nodig?
# for axis in range(3):
#     new_cuboid_pcl_points_axis = new_cuboid_pcl_points[:, axis]
#     new_cuboid_pcl_points_axis_sorted_indices = np.argsort(new_cuboid_pcl_points_axis)
#     new_cuboid_pcl_points_axis_sorted = new_cuboid_pcl_points_axis[new_cuboid_pcl_points_axis_sorted_indices]
#     new_cuboid_pcl_points_axis_sorted_diff = np.diff(new_cuboid_pcl_points_axis_sorted)
#     new_cuboid_pcl_points_axis_sorted_diff = np.append(new_cuboid_pcl_points_axis_sorted_diff, 0)
#     new_cuboid_pcl_points_axis_sorted_diff = np.abs(new_cuboid_pcl_points_axis_sorted_diff)
#     high_diff_points = np.where(new_cuboid_pcl_points_axis_sorted_diff > max_dist_from_neighbor)[0]

#     for p in high_diff_points:
#         points_to_remove.extend(list(new_cuboid_pcl_points_axis_sorted_indices[np.arange(p + 1, len(new_cuboid_pcl_points_axis_sorted_indices))])) 

# new_cuboid_pcl = new_cuboid_pcl.select_by_index(points_to_remove, invert=True)

# # (TODO: vraag: is deze stap echt nodig?)
# new_cleaned_cuboid_pcl, ind = new_cuboid_pcl.remove_radius_outlier(nb_points=16, radius=0.01)

# o3d.io.write_point_cloud(f"{fileName}_12_newCuboidAfterRadiusRemoval.ply", new_cuboid_pcl)
# o3d.io.write_point_cloud(f"{fileName}_13_cleanedCuboid.ply", new_cleaned_cuboid_pcl)

In [ ]:
def selectClosestCluster(pcd, top_plane_point):
    pcdPoints = np.asarray(pcd.points)
    clusterLabels = np.asarray(pcd.cluster_dbscan(eps=0.01, min_points=10))

    nonNoisePoints = pcdPoints[clusterLabels != -1]
    nonNoiseClusterLabels = clusterLabels[clusterLabels != -1]
    pointDistances = np.linalg.norm(nonNoisePoints - np.asarray(top_plane_point), axis=1)

    closestPointIdx = np.argmin(pointDistances)
    closestPointLabel = nonNoiseClusterLabels[closestPointIdx]

    new_cuboid_pcl = o3d.geometry.PointCloud()
    new_cuboid_pcl.points = o3d.utility.Vector3dVector(pcdPoints[clusterLabels == closestPointLabel])

    return new_cuboid_pcl

new_cuboid_pcl = selectClosestCluster(new_cuboid_pcl, top_plane_point)
# # Visualize
# o3d.io.write_point_cloud(f"{fileName}_13_newCuboidPcl.ply", new_cuboid_pcl)

\texttt{selectClosestCluster}: Is een functie die zoekt naar een cluster datapunten die het dichts bij het gegeven \texttt{top\_plane\_point} ligt.
\\
\texttt{new\_cuboid\_pcl}: Bevat het resultaat van de \texttt{selectClosestCluster}. Het gegeven punt en de gevonden pointcloud object van de cuboid worden meegegeven als inputparameters. De \texttt{clusterlabels} zijn het resultaat van een DBSCAN-clustering op de input pointcloud. Deze wordt geconverteerd naar een numpy array. Vervolgens worden de punten en labels die door het DBSCAN-algoritme als ruis werden beschouwd geselecteerd. Dit gebeurt door de punten te selecteren waarvan de labels niet gelijk zijn aan -1. Deze punten worden opgeslagen in de variabelen \texttt{nonNoisePoints} en \texttt{nonNoiseClusterLabels}. 
\\
De functie berekent daarna de afstand van elk punt in \texttt{nonNoisePoints} tot \texttt{top_plane_point} met behulp van de L2-norm en slaat het resultaat op in \texttt{pointDistances}. \texttt{np.argmin()} wordt gebruikt om de index te vinden van het punt in \texttt{nonNoisePoints} dat de minimale afstand heeft tot \texttt{top_plane_point} en slaat het op in de variabele \texttt{closerPointIdx}. Vervolgens worden de labels van het dichtstbijzijnde punt gevonden door het overeenkomstige label te selecteren uit \texttt{nonNoiseClusterLabels} en slaat het op in de variabele \texttt{closestPointLabel}.
\\
De functie maakt daaropvolgend een nieuw puntenwolkobject met de naam \texttt{new\_cuboid\_pcl} en wijst er alleen de punten van de oorspronkelijke puntenwolk aan toe waarvan de labels overeenkomen met het label \texttt{closestPointLabel}. Dit geeft als resultaat een puntenwolk die alleen de punten bevat die behoren tot de cluster die het dichtst bij \texttt{top\_plane\_point} ligt.
\\
Ten slotte retourneert de functie het nieuwe gefilterde puntenwolkobject \texttt{new\_cuboid\_pcl}

# 9. Begrenzingskader rond cuboid passen

In deze stap zullen we een begrenzingskader of bounding box rond de overgebleven punten van de cuboid passen.
Een begrenzingskader is een vierkant dat rond een object wordt getrokken om dit object te omsluiten. Het kan gebruikt worden om het minimale gebied te identificeren die het object bevat. Dit zal ons later helpen bij de opmeting van de doos.

In [ ]:
# get axis aligned bounding box
bb = new_cuboid_pcl.get_axis_aligned_bounding_box()
bb.color = [0, 1, 0]

# # TODO: vraag: is deze stap echt nodig?
# bb.get_extent()

\texttt{bb}: In deze variable word een axis aligned bounding box ofwel een as-uitgelijnd begrenzingskader (AABB) opgeslagen. De term "as-uitgelijnd" verwijst naar het feit dat de randen van het begrenzingskader evenwijdig zijn aan de assen van het coördinatensysteem, in plaats van onder een hoek of geroteerd te zijn. Dit type begrenzingskaders is eenvoudig en efficiënt te berekenen en te gebruiken, maar het kan leiden tot een groter begrenzingskader dan nodig is, vooral als de vorm van het object complex of geroteerd is. Dit is ook de reden waarom we onze originele cuboid geroteerd hebben in de vorige stappen.
\\
Dit is een begrenzingskader dat wordt gedefinieerd door twee punten die de minimale en maximale omvang van het kader langs elke as vertegenwoordigen. Met andere woorden, het is een rechthoekige doos die is uitgelijnd met de XYZ-assen van het coördinatensysteem, en het kan worden gebruikt om een 3D-object of een reeks punten te begrenzen.
\\
\texttt{get_axis_aligned_bounding_box()}: Deze functie wordt gebruikt om een AABB te berekenen voor een puntenwolk. Om de AABB te construeren, herhaalt het alle punten van de puntenwolk en vindt het de min/max-waarde van de x-, y- en z-coördinaten van de puntenwolk. Vervolgens worden de min/max-waarden gebruikt om de AABB te construeren, waarbij de hoek die de minimumwaarden vertegenwoordigt in x,y,z-coördinaten het punt is met de laagste coördinaten, en de hoek die de maximumwaarden vertegenwoordigt in x,y,z-coördinaten is het punt met de hoogste coördinaten.
\\
Om de gevonden begrenzingskader rond onze geroteerde cuboid terug te roteren naar het originele coordinatenstelsel van de input pointcloud gebruiken we volgende code:

In [ ]:
# translate bounding box to original coordinate system
# TODO: bb_points misschien opsplitsen in 2 variabelen
bb_points = np.asarray(bb.get_box_points())
bb_points = np.dot(np.linalg.inv(base_change_matrix), bb_points.T).T



\texttt{np.asarray(bb.get_box_points())}: Haalt de coördinaten op van de 8 hoekpunten van de AABB en slaat deze op in \texttt{bb_points}. Dit is een 8x3 matris met de 8 hoekpunten en 3 coördinaten(x,y,z) per punt.
\\
\texttt{bb_points = np.dot(np.linalg.inv(base_change_matrix), bb_points.T).T}: Deze lijn berekent het puntproduct van de inverse van \texttt{base_change_matrix} (3x3 matrix) en de transponering van \texttt{bb_points} (8x3 matrix). Dit betekent dat het een matrixvermenigvuldiging van deze twee matrices uitvoert, wat resulteert in een 8x3 matrix. Vervolgens wordt de transponering opnieuw toegepast met \texttt{.T} op het resultaat van het puntproduct, wat resulteert in een 3x8 matrix. We doen dit omdat open3d de gegevens op deze manier verwacht. De het eerste getal van de matrix moet het aantal waarnemingen vertegnwoordigen, de tweede de dimensie.
\\
Het resultaat hiervan zijn de coördinaten van begrenzinskader die zich in hetzelfde coördinatensysteem bevinden als de oorspronkelijke input puntenwolk. We doen dit zodat we het later visueel kunnen verifiëren dat de bounding box echt rond het object is getekend. Dit helpt om te beslissen of de opmeting op een juiste manier gebeurt is.

In [ ]:

# Visualize the oriented bounding box
# Create an oriented bounding box from the transformed points
new_bb = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bb_points))
new_bb.color = [0, 1, 0]


width = bb.max_bound[0] - bb.min_bound[0]
height = bb.max_bound[1] - bb.min_bound[1] + 0.02
depth = bb.max_bound[2] - bb.min_bound[2]

print(f"Width: {width}, Height: {height}, Depth: {depth}")

# o3d.visualization.draw_geometries([cloud, new_bb])


\texttt{new_bb}: Dit is een nieuwe puntenwolk die we gaan gebruiken om de begrenzingskader te visualiseren. We gebruiken de coördinaten van de 8 hoekpunten van de begrenzingskader die we net hebben berekend en we geven deze mee aan de functie \texttt{create\_from\_points_()} om een nieuwe puntenwolk te maken van het gevonden begrenzingskader. We geven ook de kleur mee die we willen gebruiken om de begrenzingskader te visualiseren. 

\texttt{min_bound} van een AxisAlignedBoundingBox-object komt overeen met de linkeronderhoek van het selectiekader. De eigenschap max_bound retourneert de tegenovergestelde hoek van het selectiekader, de rechterbovenhoek van het selectiekader.

De breedte van \texttt{bb} wordt berekend door de minimale x-coördinaat af te trekken van de maximale x-coördinaat, de hoogte van \texttt{bb} wordt berekend door de minimale y-coördinaat af te trekken van de maximale y-coördinaat en er 0,02 bij op te tellen (dat is de eerder ingestelde drempel) en de diepte van \texttt{bb} wordt berekend door de minimale z-coördinaat af te trekken van de maximale z-coördinaat. Ten slotte worden de afmetingen afgedrukt met behulp van de f-string (f"Width: {width}, Height: {height}, Depth: {depth}")

In [ ]:
# o3d.visualization.draw_geometries([cloud, new_bb])